# Import libraries

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/ROBT613_BMI_Final_Project

/content/drive/MyDrive/ROBT613_BMI_Final_Project


In [3]:
!pip install mne==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313874 sha256=8ae2dac2f2e61f741498b206f2f04f6413f4d2af143825831fc2cdc018ff1422
  Stored in directory: /root/.cache/pip/wheels/50/55/a4/d95e3c6ab9370b77f9c871e9882a9578a2195c64ede88694ff
Successfully built mne


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import itertools
import pandas as pd
import pickle
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler

from nu_smrutils import loaddat,subject_specific,augment_dataset,crop_data,train_model

%matplotlib inline
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

# Select device

In [5]:
#by default
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())

Your GPU device name : Tesla T4


# Select Dataset

## 1. BNCI2014004

In [6]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


# Load subject specific data, preprocess data (filtering, data augmentation, data cropping, standard scaling), and use Pytorch dataloaders

Function for standard scaler

In [7]:
scaler = StandardScaler()
def standard_scaling(data):
  shape1 = data.shape[1]
  shape2 = data.shape[2]
  data = data.reshape(data.shape[0],-1)
  data = scaler.fit_transform(data)
  data = data.reshape(data.shape[0],shape1,shape2)
  return data

Function for Pytorch dataloaders

In [8]:
def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):
    # convert data dimensions to into to gray scale image format
    if EEGNET: ### EEGNet model requires the last dimension to be 1
        ff = lambda dat: torch.unsqueeze(dat, dim = -1)
    else:
        ff = lambda dat: torch.unsqueeze(dat, dim = 1)

    x_train, x_valid, x_test =  map(ff,(dat['xtrain'],dat['xvalid'],dat['xtest']))
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)

    # TensorDataset & Dataloader
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev))
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev))

    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader},
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})
    return output

In [9]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Data cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
to

# CNN architecture




In [10]:
class CNN(torch.nn.Module):
    def __init__(self, input_size, kernel_size, conv_channels,
                 dense_size, dropout):
        super(CNN, self).__init__()
        self.cconv   = []
        self.MaxPool = nn.MaxPool2d((1, 2), (1, 2))
        self.ReLU    = nn.ReLU()
        self.Dropout = nn.Dropout(dropout)
        self.batchnorm = []
        self.outputs   = 2 # number of output nodes or classess
        # ############ batchnorm ###########
        for jj in conv_channels:
            self.batchnorm.append(nn.BatchNorm2d(jj, eps=0.001, momentum=0.01,
                                                 affine=True, track_running_stats=True).cuda())
        ii = 0 ##### define CONV layer architecture: #####
        for in_channels, out_channels in zip(conv_channels, conv_channels[1:]):
            conv_i = torch.nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                                     kernel_size = kernel_size[ii], #stride = (1, 2),
                                     padding = (kernel_size[ii][0]//2, kernel_size[ii][1]//2))
            self.cconv.append(conv_i)
            self.add_module('CNN_K{}_O{}'.format(kernel_size[ii], out_channels), conv_i)
            ii += 1
        self.flat_dim = self.get_output_dim(input_size, self.cconv)
        self.fc1 = torch.nn.Linear(self.flat_dim, dense_size)
        self.fc2 = torch.nn.Linear(dense_size, self.outputs)

    def get_output_dim(self, input_size, cconv):
        with torch.no_grad():
            input = torch.ones(1,*input_size)
            for conv_i in cconv:
                input = self.MaxPool(conv_i(input))
                flatout = int(np.prod(input.size()[1:]))
                print("Input shape : {} and flattened : {}".format(input.shape, flatout))
        return flatout

    def forward(self, input):
        for jj, conv_i in enumerate(self.cconv):
            input = conv_i(input)
            input = self.batchnorm[jj+1](input)
            input = self.ReLU(input)
            input = self.MaxPool(input)
        # flatten the CNN output
        out = input.view(-1, self.flat_dim)
        out = self.fc1(out)
        out = self.Dropout(out)
        out = self.fc2(out)
        return out

# Training loop

In [11]:
#Hyperparameter settings
num_epochs = 75
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  kernel_size = [(3, 8), (3, 8),(3, 8),(3, 8),(3, 8),(3,8)]
  conv_chan = [1, 128, 64, 32, 16, 8]

  model = CNN(input_size    = input_size,
              kernel_size   = kernel_size,
              conv_channels = conv_chan,
              dense_size    = 256,
              dropout       = 0.5)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Input shape : torch.Size([1, 128, 3, 60]) and flattened : 23040
Input shape : torch.Size([1, 64, 3, 30]) and flattened : 5760
Input shape : torch.Size([1, 32, 3, 15]) and flattened : 1440
Input shape : torch.Size([1, 16, 3, 8]) and flattened : 384
Input shape : torch.Size([1, 8, 3, 4]) and flattened : 96
Epoch 1/75
train loss: 0.0113, acc: 0.4843
val loss: 0.0126, acc: 0.5036
Epoch 2/75
train loss: 0.0109, acc: 0.5253
val loss: 0.0126, acc: 0.5036
Epoch 3/75
train loss: 0.0106, acc: 0.5580
val loss: 0.0126, acc: 0.4928
Epoch 4/75
train loss: 0.0099, acc: 0.6276
val loss: 0.0125, acc: 0.5616
Epoch 5/75
train loss: 0.0088, acc: 0.6936
val loss: 0.0130, acc: 0.5725
Epoch 6/75
train loss: 0.0071, acc: 0.7574
val loss: 0.0134, acc: 0.5471
Epoch 7/75
train loss: 0.0060, acc: 0.7869
val loss: 0.0136, acc: 0.6087
Epoch 8/75
train loss: 0.0055, acc: 0.8010
val loss: 0.0143, acc: 0.5616
Epoch 9/75
train loss: 0.0050, acc: 0.8260
val loss: 0.0142, ac

# Save and print results

In [12]:
table.to_csv('results/CNN/BNCI2014004_cnn.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.866346  0.648551      0.68     13
Subject2   0.819416  0.616858      0.56     36
Subject3   0.989103  0.532609      0.56     31
Subject4   0.951935  0.912281      0.94      9
Subject5   0.817104  0.596491      0.58     15
Subject6   0.847756  0.699275      0.74      7
Subject7   0.910577  0.721014      0.64     19
Subject8   0.871888  0.814433      0.80      6
Subject9   0.867308  0.739130      0.72     10

Average between subjects: 0.69
Max test_score between subjects: 0.94


Repeat the same steps for other datasets
## 2. BNCI2014001

In [13]:
itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


In [14]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.000001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Data cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

In [15]:
#Hyperparameter settings
num_epochs = 75
learning_rate = 1e-3
weight_decay = 1e-3
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  kernel_size = [(3, 8), (3, 8),(3, 8),(3, 8),(3, 8),(3,8)]
  conv_chan = [1, 128, 64, 32, 16, 8]

  model = CNN(input_size    = input_size,
              kernel_size   = kernel_size,
              conv_channels = conv_chan,
              dense_size    = 256,
              dropout       = 0.5)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)


  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Input shape : torch.Size([1, 128, 22, 60]) and flattened : 168960
Input shape : torch.Size([1, 64, 22, 30]) and flattened : 42240
Input shape : torch.Size([1, 32, 22, 15]) and flattened : 10560
Input shape : torch.Size([1, 16, 22, 8]) and flattened : 2816
Input shape : torch.Size([1, 8, 22, 4]) and flattened : 704
Epoch 1/75
train loss: 0.0120, acc: 0.5330
val loss: 0.0125, acc: 0.5135
Epoch 2/75
train loss: 0.0112, acc: 0.5773
val loss: 0.0131, acc: 0.5586
Epoch 3/75
train loss: 0.0087, acc: 0.7456
val loss: 0.0163, acc: 0.4955
Epoch 4/75
train loss: 0.0062, acc: 0.8205
val loss: 0.0182, acc: 0.5225
Epoch 5/75
train loss: 0.0039, acc: 0.8986
val loss: 0.0211, acc: 0.5225
Epoch 6/75
train loss: 0.0021, acc: 0.9549
val loss: 0.0200, acc: 0.5586
Epoch 7/75
train loss: 0.0009, acc: 0.9855
val loss: 0.0317, acc: 0.5315
Epoch 8/75
train loss: 0.0012, acc: 0.9734
val loss: 0.0323, acc: 0.5676
Epoch 9/75
train loss: 0.0010, acc: 0.9775
val loss: 

In [16]:
table.to_csv('results/CNN/BNCI2014001_cnn.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   1.000000  0.621622      0.54     66
Subject2   0.881643  0.576577      0.68      7
Subject3   0.963768  0.864865      0.88      5
Subject4   0.552335  0.612613      0.50      3
Subject5   1.000000  0.576577      0.46     41
Subject6   0.544283  0.666667      0.56      3
Subject7   0.557971  0.522523      0.54      3
Subject8   0.942029  0.819820      0.80     66
Subject9   0.793076  0.810811      0.88     63

Average between subjects: 0.65
Max test_score between subjects: 0.88


## 3. Weibo2014

In [17]:
itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


In [18]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Data cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
tor

In [19]:
#Hyperparameter settings
num_epochs = 75
learning_rate = 1e-3
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  kernel_size = [(3, 8), (3, 8),(3, 8),(3, 8),(3, 8),(3,8)]
  conv_chan = [1, 128, 64, 32, 16, 8]

  model = CNN(input_size    = input_size,
              kernel_size   = kernel_size,
              conv_channels = conv_chan,
              dense_size    = 256,
              dropout       = 0.5)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)


  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Input shape : torch.Size([1, 128, 60, 60]) and flattened : 460800
Input shape : torch.Size([1, 64, 60, 30]) and flattened : 115200
Input shape : torch.Size([1, 32, 60, 15]) and flattened : 28800
Input shape : torch.Size([1, 16, 60, 8]) and flattened : 7680
Input shape : torch.Size([1, 8, 60, 4]) and flattened : 1920
Epoch 1/75
train loss: 0.0257, acc: 0.4696
val loss: 0.0131, acc: 0.5079
Epoch 2/75
train loss: 0.0133, acc: 0.4841
val loss: 0.0119, acc: 0.4762
Epoch 3/75
train loss: 0.0114, acc: 0.5478
val loss: 0.0117, acc: 0.5397
Epoch 4/75
train loss: 0.0114, acc: 0.5130
val loss: 0.0116, acc: 0.4603
Epoch 5/75
train loss: 0.0112, acc: 0.5275
val loss: 0.0119, acc: 0.4444
Epoch 6/75
train loss: 0.0110, acc: 0.5377
val loss: 0.0118, acc: 0.4762
Epoch 7/75
train loss: 0.0114, acc: 0.5507
val loss: 0.0126, acc: 0.4286
Epoch 8/75
train loss: 0.0111, acc: 0.5812
val loss: 0.0117, acc: 0.4603
Epoch 9/75
train loss: 0.0111, acc: 0.5580
val loss

In [20]:
table.to_csv('results/CNN/Weibo2014_cnn.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1    0.986957  0.698413      0.54     43
Subject2    0.997101  0.714286      0.66     18
Subject3    0.607246  0.571429      0.56      6
Subject4    1.000000  0.619048      0.44     43
Subject5    0.510145  0.666667      0.52      3
Subject6    1.000000  0.814815      0.52     28
Subject7    0.998551  0.968254      0.74     32
Subject8    0.666667  0.619048      0.52     12
Subject9    0.991304  0.634921      0.66     24
Subject10   0.872464  0.666667      0.64     20

Average between subjects: 0.58
Max test_score between subjects: 0.74
